# Mapa de calor das viagens de pedestres no município de São Paulo

---

In [ ]:
import pandas as pd
import geopandas
import folium
from folium.plugins import HeatMap
from shapely.geometry import Point

Carregando os datasets

In [ ]:
od23_path = "../../data/OD/ODS/od23_all.csv" 
od = pd.read_csv(od23_path)

od_districts_shapefile_path = "../../data/OD/OD-2023/002_Site Metro Mapas_190225/Shape/Distritos_2023_region.shp" 
od_districts = geopandas.read_file(od_districts_shapefile_path)

od_zones_shapefile_path = "../../data/OD/OD-2023/002_Site Metro Mapas_190225/Shape/Zonas_2023.shp" 
od_zones = geopandas.read_file(od_zones_shapefile_path)

Filtrando zonas OD que pertencem ao município de SP

In [ ]:
od_zones = od_zones.query('NumeroMuni == 36')

Convertendo CRS

In [ ]:
od_districts = od_districts.to_crs("EPSG:4326")
od_zones = od_zones.to_crs("EPSG:4326")

In [ ]:
od_districts.head()

Filtrando viagens que originaram-se no município de São Paulo e pedestres

In [ ]:
pedestrians_sp = od.query("MUNI_O == 36 and MODOPRIN == 17") # apenas SP e pedestres

Filtrando registros com coordenadas não válidas

In [ ]:
pedestrians_coords = (
    pedestrians_sp[["CO_O_X", "CO_O_Y", "FE_VIA"]]
    .dropna()
    .query("CO_O_X != 0 and CO_O_Y != 0")  # remover coords nulas
)

Cálculo da representatividade do ponto em relação ao total de viagens a pé 

In [ ]:
total_viagens = pedestrians_coords["FE_VIA"].sum()
pedestrians_coords["weight"] = pedestrians_coords["FE_VIA"] / total_viagens

## Mapa de calor

In [ ]:
# Criando mapa base centralizado em São Paulo
sp_center = [-23.55, -46.63]
m = folium.Map(location=sp_center, zoom_start=11, tiles="CartoDB positron")

Cria mapa de calor com origens dos pedestres de SP

In [ ]:
# Adicionando camada de calor ponderada por FE_VIA
heat_data = pedestrians_coords[["CO_O_X", "CO_O_Y", "weight"]].copy()

heat_map = HeatMap(heat_data.values.tolist(), radius=8, blur=15, max_zoom=1).add_to(m)

heat_data.head()

Adiciona polígonos das zonas OD como camada interativa

In [ ]:
folium.GeoJson(
    od_zones,
    name="Zonas OD",
    style_function=lambda feature: {
        "fillColor": "transparent",
        "color": "green",
        "weight": 1,
        "fillOpacity": 0.1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["NumeroZona", "NomeZona"],
        aliases=["Zona:", "Nome:"],
        sticky=False
    )
).add_to(m)

Intersects entre pontos e distritos para remover distritos que não pertencem ao município de São Paulo

In [ ]:
# GeoDataFrame de pontos a partir das coordenadas de pedestres
gdf_points = geopandas.GeoDataFrame(
    pedestrians_coords,
    geometry=geopandas.points_from_xy(pedestrians_coords["CO_O_Y"], pedestrians_coords["CO_O_X"]),
    crs="EPSG:4326"
)

# spatial join: pontos x distritos
points_in_districts = geopandas.sjoin(gdf_points, od_districts, how="inner", predicate="intersects")

# apenas os distritos que tiveram interseção
od_districts_sp = od_districts.loc[
    od_districts.index.isin(points_in_districts["index_right"].unique())
].copy()

print(f"Número de distritos antes: {len(od_districts)}")
print(f"Número de distritos em SP (com viagens a pé): {len(od_districts_sp)}")


Remover municípios que restaram: 'Diadema', 'Guarulhos', 'Osasco', 'São Caetano do Sul'

In [ ]:
od_districts_sp = od_districts_sp.query(
    "NomeDistri not in ['Diadema', 'Guarulhos', 'Osasco', 'São Caetano do Sul']"
)

Adiciona polígonos dos distritos como camada interativa

In [ ]:
folium.GeoJson(
    od_districts_sp,
    name="Distritos",
    style_function=lambda feature: {
        "fillColor": "transparent",
        "color": "red",
        "weight": 1,
        "fillOpacity": 0.05,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["NumeroDist", "NomeDistri"],
        aliases=["Distrito:", "Nome:"],
        sticky=False
    ),
    show=False
).add_to(m)

Salva o mapa

In [ ]:
# Controle de camadas
folium.LayerControl().add_to(m)

m.save("../built_maps/choropleth_ped_trips_maps/ped_heat_map.html")